In [1]:
from ..conftest import setup_project
setup_project()

ImportError: attempted relative import with no known parent package

In [2]:
import jax
import jax.numpy as jnp
import chex
import numpy as np
#disable jax JIT
#jax.config.update("jax_disable_jit", True)

from tqdm.autonotebook import tqdm
from JAxtar.hash import hash_func_builder, HashTable
from JAxtar.bgpq import BGPQ, HashTableIdx_HeapValue
from functools import partial
from puzzle.lightsout import LightsOut
from heuristic.lightsout_heuristic import LightsOutHeuristic

/tmp/ipykernel_2547500/813126535.py:8: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [3]:
size = int(2e7)
batch_size = int(8192)
cost_weight = 1.0 - 1e-3 # for tie breaking

In [4]:
puzzle = LightsOut(7)

In [5]:
hash_func = hash_func_builder(puzzle.State)
table = HashTable.build(puzzle.State, 1, size)
size_table = table.capacity
n_table = table.n_table
heap = BGPQ.build(size, batch_size, HashTableIdx_HeapValue)
cost = jnp.full((size_table, n_table), jnp.inf)
not_closed = jnp.full((size_table, n_table), True, dtype=bool)
parent = jnp.full((size_table, n_table, 2), -1, dtype=int)

In [6]:
lookup = jax.jit(jax.vmap(partial(HashTable.lookup, hash_func), in_axes=(None, 0)))
parallel_insert = jax.jit(partial(HashTable.parallel_insert, hash_func))
heuristic = jax.jit(jax.vmap(LightsOutHeuristic(puzzle).distance, in_axes=(0, None)))
solved_fn = jax.jit(jax.vmap(puzzle.is_solved, in_axes=(0, None)))
neighbours_fn = jax.jit(jax.vmap(puzzle.get_neighbours, in_axes=(0,0)))
delete_fn = jax.jit(BGPQ.delete_mins)
insert_fn = jax.jit(BGPQ.insert)

In [7]:
states = jax.vmap(puzzle.get_initial_state, in_axes=0)(key=jax.random.split(jax.random.PRNGKey(128),1))
#states = puzzle.State(board=jnp.array([0, 12, 9, 13, 15, 11, 10, 14, 3, 7, 2, 5, 4, 8, 6, 1], dtype=jnp.uint8))[jnp.newaxis, ...]
# 80 moves to solve
target = puzzle.get_target_state()
states, filled = HashTable.make_batched(puzzle.State, states, batch_size)
cost_val = jnp.full((batch_size,), jnp.inf).at[0].set(0)
table, inserted, idx, table_idx = parallel_insert(table, states, filled)
found = inserted
print(found[:10])
heur_val = heuristic(states, target)
print(heur_val[:10])
hash_idxs = HashTableIdx_HeapValue(index=idx, table_index=table_idx)[:, jnp.newaxis]
cost = cost.at[idx, table_idx].set(jnp.where(found, cost_val, cost[idx, table_idx]))
key = cost_val + heur_val
heap = BGPQ.insert(heap, key, hash_idxs)

[ True False False False False False False False False False]
[3.  9.8 9.8 9.8 9.8 9.8 9.8 9.8 9.8 9.8]


In [8]:
print(states[0])
pbar = tqdm(total=size)
pbar.update(1)
while heap.size < size and not heap.size == 0 and table.size < size:
    pbar_str = f"heap_size: {heap.size:8d}, total_nodes: {table.size:8d}, "
    # get the minimum key
    heap, min_key, min_val = delete_fn(heap)
    min_idx, min_table_idx = min_val.index, min_val.table_index
    parent_idx = jnp.stack((min_idx, min_table_idx), axis=-1).astype(jnp.int32)
    # get the state
    cost_val, not_closed_val = cost[min_idx, min_table_idx], not_closed[min_idx, min_table_idx]
    states = table.table[min_idx, min_table_idx]
    solved = solved_fn(states, target)
    if solved.any():
        break

    # check if the state is already closed
    filled = jnp.logical_and(jnp.isfinite(min_key), not_closed_val)
    if not filled.any():
        continue
    not_closed = not_closed.at[min_idx, min_table_idx].min(~filled)
    closed_ratio = jnp.mean(~filled)
    pbar_str += f"cost: {jnp.mean(cost_val):.2f}, closed_ratio: {closed_ratio:.2f}"
    pbar.set_description_str(pbar_str)

    neighbours, ncost = neighbours_fn(states, filled)
    nextcosts = cost_val[:, jnp.newaxis] + ncost
    nextheur = jax.vmap(heuristic, in_axes=(0, None))(neighbours, target)
    nextkeys = cost_weight * nextcosts + nextheur
    filleds = jnp.isfinite(nextkeys)
    for i in range(nextkeys.shape[1]):
        nextkey = nextkeys[:, i]
        nextcost = nextcosts[:, i]
        nextstates = neighbours[:, i]
        filled = filleds[:, i]

        table, inserted, idx, table_idx = parallel_insert(table, nextstates, filled)
        vals = HashTableIdx_HeapValue(index=idx, table_index=table_idx)[:, jnp.newaxis]
        optimal = jnp.less(nextcost, cost[idx, table_idx])
        cost = cost.at[idx, table_idx].min(nextcost)
        parent = parent.at[idx, table_idx].set(jnp.where(optimal[:,jnp.newaxis], parent_idx, parent[idx, table_idx]))
        not_closed_update = not_closed[idx, table_idx] | optimal
        not_closed = not_closed.at[idx, table_idx].set(not_closed_update)
        nextkey = jnp.where(not_closed_update, nextkey, jnp.inf)
        added = int(jnp.sum(optimal))
        
        heap = insert_fn(heap, nextkey, vals, added_size=added)
        pbar.update(added)
pbar.close()
solved_st = states[solved][0]
n_cost = cost_val[solved][0]
print(solved_st)
print(n_cost)

┏━━━━━━━━━━━━━━━┓
┃ □ □ □ □ □ □ ■ ┃
┃ □ □ □ □ □ ■ ■ ┃
┃ □ ■ □ □ □ □ ■ ┃
┃ ■ ■ ■ □ □ □ □ ┃
┃ ■ □ ■ □ □ □ □ ┃
┃ □ ■ □ □ □ □ □ ┃
┃ ■ ■ □ ■ ■ □ □ ┃
┗━━━━━━━━━━━━━━━┛


heap_size:   568214, total_nodes:   587000, cost: 4.49, closed_ratio: 0.17:   4%|▍         | 850051/20000000 [00:10<04:01, 79316.44it/s] 


┏━━━━━━━━━━━━━━━┓
┃ □ □ □ □ □ □ □ ┃
┃ □ □ □ □ □ □ □ ┃
┃ □ □ □ □ □ □ □ ┃
┃ □ □ □ □ □ □ □ ┃
┃ □ □ □ □ □ □ □ ┃
┃ □ □ □ □ □ □ □ ┃
┃ □ □ □ □ □ □ □ ┃
┗━━━━━━━━━━━━━━━┛
6.0


In [9]:
path = []
idx = jnp.argmax(jnp.max(solved, axis=0))
parent_last = parent_idx[idx]
for i in range(100):
    if parent_last[0] == -1:
        break
    path.append(parent_last)
    parent_last = parent[parent_last[0], parent_last[1]]

for p in path[::-1]:
    state = table.table[p[0], p[1]]
    c = cost[p[0], p[1]]
    print(state)
    print(c)
print(solved_st)
print(n_cost)

┏━━━━━━━━━━━━━━━┓
┃ □ □ □ □ □ □ ■ ┃
┃ □ □ □ □ □ ■ ■ ┃
┃ □ ■ □ □ □ □ ■ ┃
┃ ■ ■ ■ □ □ □ □ ┃
┃ ■ □ ■ □ □ □ □ ┃
┃ □ ■ □ □ □ □ □ ┃
┃ ■ ■ □ ■ ■ □ □ ┃
┗━━━━━━━━━━━━━━━┛
0.0
┏━━━━━━━━━━━━━━━┓
┃ □ □ □ □ □ □ □ ┃
┃ □ □ □ □ □ □ □ ┃
┃ □ ■ □ □ □ □ □ ┃
┃ ■ ■ ■ □ □ □ □ ┃
┃ ■ □ ■ □ □ □ □ ┃
┃ □ ■ □ □ □ □ □ ┃
┃ ■ ■ □ ■ ■ □ □ ┃
┗━━━━━━━━━━━━━━━┛
1.0
┏━━━━━━━━━━━━━━━┓
┃ □ □ □ □ □ □ □ ┃
┃ □ □ □ □ □ □ □ ┃
┃ □ ■ □ □ □ □ □ ┃
┃ ■ ■ ■ □ □ □ □ ┃
┃ ■ □ ■ □ □ □ □ ┃
┃ □ ■ □ ■ □ □ □ ┃
┃ ■ ■ ■ □ □ □ □ ┃
┗━━━━━━━━━━━━━━━┛
2.0
┏━━━━━━━━━━━━━━━┓
┃ □ □ □ □ □ □ □ ┃
┃ □ □ □ □ □ □ □ ┃
┃ □ ■ □ □ □ □ □ ┃
┃ ■ ■ ■ □ □ □ □ ┃
┃ ■ □ □ □ □ □ □ ┃
┃ □ □ ■ □ □ □ □ ┃
┃ ■ ■ □ □ □ □ □ ┃
┗━━━━━━━━━━━━━━━┛
3.0
┏━━━━━━━━━━━━━━━┓
┃ □ □ □ □ □ □ □ ┃
┃ □ □ □ □ □ □ □ ┃
┃ □ ■ □ □ □ □ □ ┃
┃ ■ ■ ■ □ □ □ □ ┃
┃ ■ ■ □ □ □ □ □ ┃
┃ ■ ■ □ □ □ □ □ ┃
┃ ■ □ □ □ □ □ □ ┃
┗━━━━━━━━━━━━━━━┛
4.0
┏━━━━━━━━━━━━━━━┓
┃ □ □ □ □ □ □ □ ┃
┃ □ □ □ □ □ □ □ ┃
┃ □ □ □ □ □ □ □ ┃
┃ □ □ □ □ □ □ □ ┃
┃ ■ □ □ □ □ □ □ ┃
┃ ■ ■ □ □ □ □ □ ┃
┃ ■ □ □ □ □ □ □ ┃
┗━━━━━━━━━━━━━━━┛
5.0
┏━━━